In [19]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

In [2]:
path = 'Asset/tedsd_puf_2019.dta'
df_discharge_2019 = pd.read_stata(path)

In [3]:
state_cluster_0 = [ 1.0,  4.0,  5.0,  6.0,  8.0,  9.0, 10.0, 13.0, 15.0, 17.0, 18.0,
              19.0, 20.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0,
              31.0, 32.0, 34.0, 36.0, 39.0, 40.0, 44.0, 45.0, 46.0, 47.0, 48.0,
              49.0, 50.0, 55.0, 72.0]

In [4]:
def dataset_on_filter(df, filter_one:dict = None, filter_two:dict = None, filter_three:dict = None):
    df_processing = df.copy()

    if filter_one:
        for value in filter_one:
            df_processing = df_processing[df_processing[value] == filter_one[value]]

    if filter_two:
        for value in filter_two:
            df_processing = df_processing[df_processing[value] == filter_two[value]]

    if filter_three:
        for value in filter_three:
            df_processing = df_processing[df_processing[value] == filter_three[value]]

    df_filtered = df_processing.copy()
    name_str = '{}_{}_{}'.format(filter_one, filter_two, filter_three)
    df_filtered.name = name_str
    return df_filtered

In [5]:
def treatment_variable_transform(df, drop_value:list = None):
    df_processing = df.copy()

    # Drop certain record with REASON equals certain value(s)
    if drop_value:
        for value in drop_value:
            df_processing.drop(df_processing.index[df_processing['REASON'] == value], inplace=True)
    
    # Create the Treatment variable column with value 1 and 0 only
    df_processing['Treatment_Completed'] = (df_processing.REASON == 1)
    df_processing.replace(True, 1, inplace = True)
    df_processing.replace(False, 0,  inplace = True)

    df_transformed = df_processing
    return df_transformed

In [32]:
df_cluster_0 = df_discharge_2019.loc[df_discharge_2019['STFIPS'].isin(state_cluster_0)]

In [34]:
corrs = df_cluster_0[[ 'EDUC', 'MARSTAT', 'SERVICES',
       'DETCRIM', 'LOS', 'PSOURCE', 'NOPRIOR', 'ARRESTS', 'EMPLOY', 'METHUSE',
       'PSYPROB', 'LIVARAG', 'DAYWAIT',
       'REASON',  'DSMCRIT', 'AGE',
       'RACE', 'ETHNIC', 'DETNLF', 'PRIMINC', 'SUB1', 'ROUTE1',
       'FREQ1', 'FRSTUSE1', 'HLTHINS', 'PRIMPAY', 'FREQ_ATND_SELF_HELP',
       'IDU', 'ALCDRUG']]

In [35]:
target = treatment_variable_transform(df_cluster_0).Treatment_Completed

In [36]:
X_train, X_test, y_train, y_test = train_test_split(corrs, target, test_size=0.20, random_state=42)


In [37]:
model = RandomForestClassifier(random_state=42)
param_grid = { 
    'n_estimators': [105, 110, 115],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [8,9,10],
    'criterion' :['gini', 'entropy']
}

CV_rfc = GridSearchCV(estimator=model, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
CV_rfc.best_params_

In [ ]:
# model = RandomForestClassifier(criterion =  ,
#                                max_depth =  ,
#                                max_features = ,
#                                n_estimators = , 
#                                random_state=42)

# model.fit(X_train, y_train)

In [ ]:
# print('Test Accuracy: {:.2f}'.format(model.score(X_test, y_test)))
# print('Test AUC: {:.2f}'.format(roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])))